In [1]:
import pandas as pd
import numpy as np

# Data loading

### Gene-expression-profiles

In [2]:
brca_ex = pd.read_hdf('../data/TCGA_gene_exp_20k_std-MAD.h5', key='brca')

In [3]:
non_brca_ex = pd.read_hdf('../data/TCGA_gene_exp_20k_std-MAD.h5', key='non_brca')

In [4]:
import ast
with open('../common_genes_1-5.txt','r') as f:
   common_genes = ast.literal_eval(f.read())

In [5]:
brca_ex = brca_ex[list(common_genes)]

In [6]:
non_brca_ex = non_brca_ex[list(common_genes)]

In [7]:
brca_ex.shape

(1212, 1492)

In [8]:
non_brca_ex.shape

(9285, 1492)

### Overall Survival

In [9]:
brca_os = pd.read_hdf('../data/TCGA_data.h5', key='brca_clinical')[['OS', 'OS.time']]

In [10]:
## brca
brca_os['OS.time'] = brca_os['OS.time'].map(lambda x: np.nan if x == 'NaN' else x)
brca_os['OS'] = brca_os['OS'].map(lambda x: np.nan if x == 'NaN' else x)
brca_os.dropna(subset=['OS.time', 'OS'], inplace=True)
brca_os['OS.time'] = brca_os['OS.time'].astype(float)
brca_os['OS'] = brca_os['OS'].astype(float)

In [11]:
brca_os.shape

(1196, 2)

In [12]:
brca_ex = brca_ex.loc[brca_os.index]

In [13]:
sum(brca_ex.index == brca_os.index)

1196

In [14]:
non_brca_os = pd.read_hdf('../data/TCGA_data.h5', key='non_brca_clinical')[['OS', 'OS.time']]

In [15]:
## brca
non_brca_os['OS.time'] = non_brca_os['OS.time'].map(lambda x: np.nan if x == 'NaN' else x)
non_brca_os['OS'] = non_brca_os['OS'].map(lambda x: np.nan if x == 'NaN' else x)
non_brca_os.dropna(subset=['OS.time', 'OS'], inplace=True)
non_brca_os['OS.time'] = non_brca_os['OS.time'].astype(float)
non_brca_os['OS'] = non_brca_os['OS'].astype(float)

In [16]:
non_brca_os.shape

(9224, 2)

In [17]:
non_brca_ex = non_brca_ex.loc[non_brca_os.index]

In [18]:
sum(non_brca_ex.index == non_brca_os.index)

9224

##### From OS to survival vector

In [19]:
import nnet_survival

Using TensorFlow backend.


In [20]:
# Create 39 equally spaced intervals for follow-up time, from 0 to 5 years:
breaks = np.arange(0.,365.*5,365./8)
n_intervals = len(breaks)-1
timegap = breaks[1:] - breaks[:-1]

In [21]:
brca_y = nnet_survival.make_surv_array(brca_os['OS.time'],
                             brca_os['OS'],
                             breaks)
non_brca_y = nnet_survival.make_surv_array(non_brca_os['OS.time'],
                             non_brca_os['OS'],
                             breaks)

# TRAINING MODELS

In [22]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Input, Dense, Conv1D, MaxPool1D, Activation, Dropout, Flatten
from keras import regularizers, optimizers
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
import warnings
from keras.regularizers import l1
from keras.optimizers import Adam
import tensorflow as tf

In [23]:
# Using C-index as evaluation metric (using a custom callback)

from lifelines.utils import concordance_index
from keras.callbacks import Callback

class CIndex(Callback):
    """
    Callback that computes the C-index metric both on training and test data after each epoch.
    
    Arguments:
        train_x: Numpy array containing the training dataset.
        train_time: Numpy array with survival time for training dataset.
        train_event: Numpy array with survival event for training dataset.
        val_x: Numpy array containing the test dataset.
        val_time: Numpy array with survival time for test dataset.
        val_event: Numpy array with survival event for test dataset.
        filepath: path to save the model.
        years: number of years to compute c-index
        patience: patience for early-stop training
    """
    
    def __init__(self, train_x, train_time, train_event, val_x, val_time, val_event, filepath, years=2, patience=5):
        super(Callback, self).__init__()
        self.X_tr = train_x
        self.time_tr = train_time
        self.event_tr = train_event
        self.X_val = val_x
        self.time_val = val_time
        self.event_val = val_event
        self.years = years
        self.patience = patience
        self.best = 0.0
        self.wait = 0  #counter for patience
        self.best_rounds = 1
        self.counter = 0
        self.filepath = filepath
        
    def on_train_begin(self, logs={}):
        self.c_index_dict = {'c-index': [], 'val_c-index': []}

    def on_epoch_end(self, epoch, logs={}):
        self.counter +=1

        y_tr=self.model.predict_proba(self.X_tr,verbose=0)
        yr_surv=np.cumprod(y_tr[:,0:np.nonzero(breaks>365*self.years)[0][0]], axis=1)[:,-1]
        c_index_tr = concordance_index(self.time_tr, yr_surv, self.event_tr)
        y_val=self.model.predict_proba(self.X_val,verbose=0)
        yr_surv_val=np.cumprod(y_val[:,0:np.nonzero(breaks>365*self.years)[0][0]], axis=1)[:,-1]
        c_index_val = concordance_index(self.time_val, yr_surv_val, self.event_val)

        #print('\tc-index: %s - val_c-index: %s' % (str(round(c_index_tr, 4)), str(round(c_index_val, 4))))
        
        if (c_index_val > self.best):
            self.c_index_dict['c-index'].append(c_index_tr)
            self.c_index_dict['val_c-index'].append(c_index_val)

            self.best = c_index_val
            self.best_rounds = self.counter
            self.wait = 0
            self.model.save_weights(self.filepath, overwrite=True)
        else:
            if self.wait >= self.patience: #no more patience, retrieve best model
                self.model.stop_training = True
                print('Best number of rounds: %d \nValidation C-Index: %f \n' % (self.best_rounds, self.best))
                self.model.load_weights(self.filepath)
            self.wait += 1

# Random 3

In [24]:
index = np.arange(brca_ex.shape[1])
np.random.seed(seed=3)
np.random.shuffle(index)
index

array([  70,  804,  269, ...,  968,  952, 1273])

## Non-BRCA

In [ ]:
non_brca_ex = non_brca_ex[non_brca_ex.columns[index]]

In [ ]:
X_non_brca = non_brca_ex.values 

In [ ]:
X_non_brca = np.expand_dims(X_non_brca, axis=2)

In [ ]:
X_non_brca.shape

### Bayesian Optimization (non-brca)

In [ ]:
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
# BRCA
# Split dataset intor train and test
from sklearn.model_selection import train_test_split

X_non_brca_train, X_non_brca_test, y_non_brca_train, y_non_brca_test, train_index, test_index  = train_test_split(X_non_brca, non_brca_y, np.arange(len(X_non_brca)), test_size=0.2, random_state=42)
X_non_brca_train, X_non_brca_val, y_non_brca_train, y_non_brca_val, train_index, val_index = train_test_split(X_non_brca_train, y_non_brca_train, train_index, test_size=0.2, random_state=44)

In [ ]:
y_non_brca_train = np.array(y_non_brca_train, dtype=int)
y_non_brca_test = np.array(y_non_brca_test, dtype=int)
y_non_brca_val = np.array(y_non_brca_val, dtype=int)

In [ ]:
print("Dataset size:", X_non_brca_train.shape, X_non_brca_test.shape, X_non_brca_val.shape)

In [ ]:
space = {
    'conv_choice': hp.choice('conv_num_layers', 
                            [{'layers': 'two',
                             'units_2conv2': hp.choice('units_2conv2', [16, 32, 64, 128]),
                              '2kernel2': hp.choice('2kernel2', [3, 5, 10, 20]),
                              '2pool2': hp.choice('2pool2', [2, 3, 5]),
                             'dropout_2conv2': hp.choice('dropout_2conv2', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3conv2': hp.choice('units_3conv2', [16, 32, 64, 128]),
                             '3kernel2': hp.choice('3kernel2', [3, 5, 10, 20]),
                             '3pool2': hp.choice('3pool2', [2, 3, 5]),
                             'dropout_3conv2': hp.choice('dropout_3conv2', [0.25, 0.5, 0.75]),
                             'units_3conv3': hp.choice('units_3conv3', [16, 32, 64, 128]),
                             '3kernel3': hp.choice('kernel3', [3, 5, 10, 20]),
                             '3pool3': hp.choice('3pool3', [2, 3, 5]),
                             'dropout_3conv3': hp.choice('dropout_3conv3', [0.25, 0.5, 0.75])
                            }]
                            ),
    'dense_choice': hp.choice('dense_num_layers', 
                            [{'layers': 'two',
                             'units_2dense2': hp.choice('units_2dense2', [16, 32, 64, 128]),
                             'dropout_2dense2': hp.choice('dropout_2dense2', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3dense2': hp.choice('units_3dense2', [16, 32, 64, 128]),
                             'dropout_3dense2': hp.choice('dropout_3dense2', [0.25, 0.5, 0.75]),
                             'units_3dense3': hp.choice('units_3dense3', [16, 32, 64, 128]),
                             'dropout_3dense3': hp.choice('dropout_3dense3', [0.25, 0.5, 0.75])
                            }]
                            ),
    'units_conv1': hp.choice('units_conv1', [16, 32, 64, 128]),
    'kernel1': hp.choice('kernel1', [3, 5, 10, 20]),
    'pool1': hp.choice('pool1', [2, 3, 5]),
    'dropout_conv1': hp.choice('dropout_conv1', [0.25, 0.5, 0.75]),
    'units_dense1': hp.choice('units_dense1', [16, 32, 64, 128]),
    'dropout_dense1': hp.choice('dropout_dense1', [0.25, 0.5, 0.75]),
    'batch_size' : hp.choice('batch_size', [16, 32, 64, 128]),
    'nb_epochs' : 100,
    'lr': hp.loguniform('lr', np.log(0.0001), np.log(0.01)),
    'activation': 'relu'                                            
}

In [ ]:
def train_non_brca(params):
    print('Params testing: ', params)
    print('\n')
    
    # Prevent GPU memory allocation problems
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    K.set_session(tf.Session(config=config))

    ## CREATE MODEL    
    model = Sequential()

    # Conv layer 1 ---
    model.add(Conv1D(filters=params['units_conv1'], 
                     kernel_size=params['kernel1'],
                     input_shape = (X_non_brca_train.shape[1], X_non_brca_train.shape[2])))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool1D(pool_size=params['pool1']))
    model.add(Dropout(params['dropout_conv1']))

    if params['conv_choice']['layers'] == 'two':
        # Conv layer 2 ---
        model.add(Conv1D(filters=params['conv_choice']['units_2conv2'], 
                     kernel_size=params['conv_choice']['2kernel2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool1D(pool_size=params['conv_choice']['2pool2']))
        model.add(Dropout(params['conv_choice']['dropout_2conv2']))
    elif params['conv_choice']['layers'] == 'three':
        # Conv layer 2 ---
        model.add(Conv1D(filters=params['conv_choice']['units_3conv2'], 
                     kernel_size=params['conv_choice']['3kernel2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool1D(pool_size=params['conv_choice']['3pool2']))
        model.add(Dropout(params['conv_choice']['dropout_3conv2']))
        # Conv layer 3 ---
        model.add(Conv1D(filters=params['conv_choice']['units_3conv3'], 
                     kernel_size=params['conv_choice']['3kernel3']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool1D(pool_size=params['conv_choice']['3pool3']))
        model.add(Dropout(params['conv_choice']['dropout_3conv3']))

    # Flatten layer ---
    model.add(Flatten())
    
    # Dense layer 1 ---
    model.add(Dense(params['units_dense1']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout_dense1']))

    if params['dense_choice']['layers']== 'two':
        # Dense layer 2 ---
        model.add(Dense(params['dense_choice']['units_2dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_2dense2']))
    elif params['dense_choice']['layers']== 'three':
        # Dense layer 2 ---
        model.add(Dense(params['dense_choice']['units_3dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_3dense2']))
        # Dense layer 3 ---
        model.add(Dense(params['dense_choice']['units_3dense3']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_3dense3']))
        
    # Output layer ---
    model.add(Dense(n_intervals, activation='sigmoid'))

    optimizer = Adam(lr=params['lr'])

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizer)

    hist_c_index = CIndex(X_non_brca_train,
                        non_brca_os.iloc[train_index]['OS.time'],
                        non_brca_os.iloc[train_index]['OS'],
                        X_non_brca_val,
                        non_brca_os.iloc[val_index]['OS.time'],
                        non_brca_os.iloc[val_index]['OS'],
                        filepath = 'keras-models/non_brca_random3.h5')

    callbacks = [hist_c_index]   

    batch = params['batch_size']
    n_epoch = params['nb_epochs']

    start = timer()
    model.fit(X_non_brca_train, 
              y_non_brca_train, 
              batch_size=batch, 
              epochs=n_epoch, 
              validation_data = [X_non_brca_val, y_non_brca_val],
              callbacks=callbacks,
              verbose=0)
    run_time = timer() - start

    predictions = model.predict_proba(X_non_brca_test,verbose=0)
    twoyr_surv=np.cumprod(predictions[:,0:np.nonzero(breaks>365*2)[0][0]], axis=1)[:,-1]
    score = concordance_index(non_brca_os.iloc[test_index]['OS.time'], twoyr_surv, non_brca_os.iloc[test_index]['OS'])
    
    return {'loss': score*(-1),
            'params': params,
            'train_time': run_time,
            'status': STATUS_OK, 
            'rounds':hist_c_index.best_rounds}

In [ ]:
from timeit import default_timer as timer
from multiprocessing import Pool
from hyperopt import STATUS_OK

def objective_non_brca(params):
    from sklearn.metrics import roc_auc_score
    
    from keras.models import Sequential
    from keras.layers import Input, Dense, Conv1D, MaxPool1D, Activation, Dropout, Flatten
    from keras import regularizers, optimizers
    from keras.layers.normalization import BatchNormalization
    from keras.callbacks import EarlyStopping

    from keras.regularizers import l1
    from keras.optimizers import Adam
   
    with Pool(1) as p:
            trial = p.apply(train_non_brca, args=(params, ))

    return trial

In [ ]:
%%time

trials_non_brca = Trials()

best_non_brca = fmin(objective_non_brca, space, algo=tpe.suggest, trials=trials_non_brca, max_evals=50)

print (best_non_brca)
print (trials_non_brca.best_trial)

import pickle

with open("keras-models/non_brca_random3.pkl", 'wb') as f:
    pickle.dump([trials_non_brca, best_non_brca], f)

In [ ]:
import pickle

with open("keras-models/non_brca_random3.pkl", 'rb') as f:
    [trials_non_brca, best_non_brca] = pickle.load(f)

## BRCA

In [25]:
brca_ex = brca_ex[brca_ex.columns[index]]

In [26]:
X_brca = brca_ex.values 

In [27]:
X_brca = np.expand_dims(X_brca, axis=2)

In [28]:
X_brca.shape

(1196, 1492, 1)

### Bayesian Optimization (brca)

In [29]:
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

In [30]:
# BRCA
# Split dataset intor train and test
from sklearn.model_selection import train_test_split

X_brca_train, X_brca_test, y_brca_train, y_brca_test, train_index, test_index  = train_test_split(X_brca, brca_y, np.arange(len(X_brca)), test_size=0.2, random_state=42)
X_brca_train, X_brca_val, y_brca_train, y_brca_val, train_index, val_index = train_test_split(X_brca_train, y_brca_train, train_index, test_size=0.2, random_state=44)

In [31]:
y_brca_train = np.array(y_brca_train, dtype=int)
y_brca_test = np.array(y_brca_test, dtype=int)
y_brca_val = np.array(y_brca_val, dtype=int)

In [32]:
print("Dataset size:", X_brca_train.shape, X_brca_test.shape, X_brca_val.shape)

Dataset size: (764, 1492, 1) (240, 1492, 1) (192, 1492, 1)


In [ ]:
space = {
    'conv_choice': hp.choice('conv_num_layers', 
                            [{'layers': 'two',
                             'units_2conv2': hp.choice('units_2conv2', [16, 32, 64, 128]),
                              '2kernel2': hp.choice('2kernel2', [3, 5, 10, 20]),
                              '2pool2': hp.choice('2pool2', [2, 3, 5]),
                             'dropout_2conv2': hp.choice('dropout_2conv2', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3conv2': hp.choice('units_3conv2', [16, 32, 64, 128]),
                             '3kernel2': hp.choice('3kernel2', [3, 5, 10, 20]),
                             '3pool2': hp.choice('3pool2', [2, 3, 5]),
                             'dropout_3conv2': hp.choice('dropout_3conv2', [0.25, 0.5, 0.75]),
                             'units_3conv3': hp.choice('units_3conv3', [16, 32, 64, 128]),
                             '3kernel3': hp.choice('kernel3', [3, 5, 10, 20]),
                             '3pool3': hp.choice('3pool3', [2, 3, 5]),
                             'dropout_3conv3': hp.choice('dropout_3conv3', [0.25, 0.5, 0.75])
                            }]
                            ),
    'dense_choice': hp.choice('dense_num_layers', 
                            [{'layers': 'two',
                             'units_2dense2': hp.choice('units_2dense2', [16, 32, 64, 128]),
                             'dropout_2dense2': hp.choice('dropout_2dense2', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3dense2': hp.choice('units_3dense2', [16, 32, 64, 128]),
                             'dropout_3dense2': hp.choice('dropout_3dense2', [0.25, 0.5, 0.75]),
                             'units_3dense3': hp.choice('units_3dense3', [16, 32, 64, 128]),
                             'dropout_3dense3': hp.choice('dropout_3dense3', [0.25, 0.5, 0.75])
                            }]
                            ),
    'units_conv1': hp.choice('units_conv1', [16, 32, 64, 128]),
    'kernel1': hp.choice('kernel1', [3, 5, 10, 20]),
    'pool1': hp.choice('pool1', [2, 3, 5]),
    'dropout_conv1': hp.choice('dropout_conv1', [0.25, 0.5, 0.75]),
    'units_dense1': hp.choice('units_dense1', [16, 32, 64, 128]),
    'dropout_dense1': hp.choice('dropout_dense1', [0.25, 0.5, 0.75]),
    'batch_size' : hp.choice('batch_size', [16, 32, 64, 128]),
    'nb_epochs' : 100,
    'lr': hp.loguniform('lr', np.log(0.0001), np.log(0.01)),
    'activation': 'relu'                                            
}

In [ ]:
def train_brca(params):
    print('Params testing: ', params)
    print('\n')
    
    # Prevent GPU memory allocation problems
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    K.set_session(tf.Session(config=config))

    ## CREATE MODEL    
    model = Sequential()

    # Conv layer 1 ---
    model.add(Conv1D(filters=params['units_conv1'], 
                     kernel_size=params['kernel1'],
                     input_shape = (X_brca_train.shape[1], X_brca_train.shape[2])))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool1D(pool_size=params['pool1']))
    model.add(Dropout(params['dropout_conv1']))

    if params['conv_choice']['layers'] == 'two':
        # Conv layer 2 ---
        model.add(Conv1D(filters=params['conv_choice']['units_2conv2'], 
                     kernel_size=params['conv_choice']['2kernel2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool1D(pool_size=params['conv_choice']['2pool2']))
        model.add(Dropout(params['conv_choice']['dropout_2conv2']))
    elif params['conv_choice']['layers'] == 'three':
        # Conv layer 2 ---
        model.add(Conv1D(filters=params['conv_choice']['units_3conv2'], 
                     kernel_size=params['conv_choice']['3kernel2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool1D(pool_size=params['conv_choice']['3pool2']))
        model.add(Dropout(params['conv_choice']['dropout_3conv2']))
        # Conv layer 3 ---
        model.add(Conv1D(filters=params['conv_choice']['units_3conv3'], 
                     kernel_size=params['conv_choice']['3kernel3']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool1D(pool_size=params['conv_choice']['3pool3']))
        model.add(Dropout(params['conv_choice']['dropout_3conv3']))

    # Flatten layer ---
    model.add(Flatten())
    
    # Dense layer 1 ---
    model.add(Dense(params['units_dense1']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout_dense1']))

    if params['dense_choice']['layers']== 'two':
        # Dense layer 2 ---
        model.add(Dense(params['dense_choice']['units_2dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_2dense2']))
    elif params['dense_choice']['layers']== 'three':
        # Dense layer 2 ---
        model.add(Dense(params['dense_choice']['units_3dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_3dense2']))
        # Dense layer 3 ---
        model.add(Dense(params['dense_choice']['units_3dense3']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_3dense3']))
        
    # Output layer ---
    model.add(Dense(n_intervals, activation='sigmoid'))

    optimizer = Adam(lr=params['lr'])

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizer)

    hist_c_index = CIndex(X_brca_train,
                        brca_os.iloc[train_index]['OS.time'],
                        brca_os.iloc[train_index]['OS'],
                        X_brca_val,
                        brca_os.iloc[val_index]['OS.time'],
                        brca_os.iloc[val_index]['OS'],
                        filepath = 'keras-models/brca_random3.h5')

    callbacks = [hist_c_index]   

    batch = params['batch_size']
    n_epoch = params['nb_epochs']

    start = timer()
    model.fit(X_brca_train, 
              y_brca_train, 
              batch_size=batch, 
              epochs=n_epoch, 
              validation_data = [X_brca_val, y_brca_val],
              callbacks=callbacks,
              verbose=0)
    run_time = timer() - start

    predictions = model.predict_proba(X_brca_test,verbose=0)
    twoyr_surv=np.cumprod(predictions[:,0:np.nonzero(breaks>365*2)[0][0]], axis=1)[:,-1]
    score = concordance_index(brca_os.iloc[test_index]['OS.time'], twoyr_surv, brca_os.iloc[test_index]['OS'])
    
    return {'loss': score*(-1),
            'params': params,
            'train_time': run_time,
            'status': STATUS_OK, 
            'rounds':hist_c_index.best_rounds}

In [ ]:
from timeit import default_timer as timer
from multiprocessing import Pool
from hyperopt import STATUS_OK

def objective_brca(params):
    from sklearn.metrics import roc_auc_score
    
    from keras.models import Sequential
    from keras.layers import Input, Dense, Conv1D, MaxPool1D, Activation, Dropout, Flatten
    from keras import regularizers, optimizers
    from keras.layers.normalization import BatchNormalization
    from keras.callbacks import EarlyStopping

    from keras.regularizers import l1
    from keras.optimizers import Adam
   
    with Pool(1) as p:
            trial = p.apply(train_brca, args=(params, ))

    return trial

In [ ]:
%%time

trials_brca = Trials()

best_brca = fmin(objective_brca, space, algo=tpe.suggest, trials=trials_brca, max_evals=50)

print (best_brca)
print (trials_brca.best_trial)

import pickle

with open("keras-models/brca_random3.pkl", 'wb') as f:
    pickle.dump([trials_brca, best_brca], f)

In [ ]:
import pickle

with open("keras-models/brca_random3.pkl", 'rb') as f:
    [trials_brca, best_brca] = pickle.load(f)

## BRCA (fine-tuning)

#### Train non-brca model with all non-brca data

In [ ]:
import pickle
with open("keras-models/non_brca_random3.pkl", 'rb') as f:
    [trials_non_brca, best_non_brca] = pickle.load(f)

In [ ]:
params = trials_non_brca.best_trial['result']['params']

In [ ]:
from timeit import default_timer as timer

# Prevent GPU memory allocation problems
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.4
K.set_session(tf.Session(config=config))

## CREATE MODEL    
model = Sequential()

# Conv layer 1 ---
model.add(Conv1D(filters=params['units_conv1'], 
                 kernel_size=params['kernel1'],
                 input_shape = (X_non_brca.shape[1], X_non_brca.shape[2])))
model.add(BatchNormalization())
model.add(Activation(params['activation']))
model.add(MaxPool1D(pool_size=params['pool1']))
model.add(Dropout(params['dropout_conv1']))

if params['conv_choice']['layers'] == 'two':
    # Conv layer 2 ---
    model.add(Conv1D(filters=params['conv_choice']['units_2conv2'], 
                 kernel_size=params['conv_choice']['2kernel2']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool1D(pool_size=params['conv_choice']['2pool2']))
    model.add(Dropout(params['conv_choice']['dropout_2conv2']))
elif params['conv_choice']['layers'] == 'three':
    # Conv layer 2 ---
    model.add(Conv1D(filters=params['conv_choice']['units_3conv2'], 
                 kernel_size=params['conv_choice']['3kernel2']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool1D(pool_size=params['conv_choice']['3pool2']))
    model.add(Dropout(params['conv_choice']['dropout_3conv2']))
    # Conv layer 3 ---
    model.add(Conv1D(filters=params['conv_choice']['units_3conv3'], 
                 kernel_size=params['conv_choice']['3kernel3']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool1D(pool_size=params['conv_choice']['3pool3']))
    model.add(Dropout(params['conv_choice']['dropout_3conv3']))

# Flatten layer ---
model.add(Flatten())

# Dense layer 1 ---
model.add(Dense(params['units_dense1']))
model.add(BatchNormalization())
model.add(Activation(params['activation']))
model.add(Dropout(params['dropout_dense1']))

if params['dense_choice']['layers']== 'two':
    # Dense layer 2 ---
    model.add(Dense(params['dense_choice']['units_2dense2']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dense_choice']['dropout_2dense2']))
elif params['dense_choice']['layers']== 'three':
    # Dense layer 2 ---
    model.add(Dense(params['dense_choice']['units_3dense2']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dense_choice']['dropout_3dense2']))
    # Dense layer 3 ---
    model.add(Dense(params['dense_choice']['units_3dense3']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dense_choice']['dropout_3dense3']))

# Output layer ---
model.add(Dense(n_intervals, activation='sigmoid'))

optimizer = Adam(lr=params['lr'])

model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizer)

batch = params['batch_size']
n_epoch = params['nb_epochs']

start = timer()
model.fit(X_non_brca, 
          non_brca_y, 
          batch_size=batch, 
          epochs=n_epoch ) #, 
          #verbose=0)
run_time = timer() - start

In [ ]:
predictions = model.predict_proba(X_non_brca,verbose=0)
twoyr_surv=np.cumprod(predictions[:,0:np.nonzero(breaks>365*2)[0][0]], axis=1)[:,-1]
score = concordance_index(non_brca_os['OS.time'], twoyr_surv, non_brca_os['OS'])

In [ ]:
score

In [ ]:
model.save('keras-models/MODEL_nonBRCA_all_data_random3.h5')

#### Bayesian optimization (BRCA finetuning)

In [33]:
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

In [34]:
space_fine_tuning = {
    'choice': hp.choice('num_layers', 
                            [{'layers': 'one'},
                            {'layers': 'two',
                             'units_2dense1': hp.choice('units_2dense1', [16, 32, 64, 128, 256]),
                             'dropout_2dense1': hp.choice('dropout_2dense1', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3dense1': hp.choice('units_3dense1', [16, 32, 64, 128, 256]),
                             'dropout_3dense1': hp.choice('dropout_3dense1', [0.25, 0.5, 0.75]),
                             'units_3dense2': hp.choice('units_3dense2', [16, 32, 64, 128, 256]),
                             'dropout_3dense2': hp.choice('dropout_3dense2', [0.25, 0.5, 0.75])
                            }]
                            ),
    'units_output': n_intervals,
    'batch_size' : hp.choice('batch_size', [16, 32, 64, 128]),
    'nb_epochs' : 100,
    'lr': hp.loguniform('lr', np.log(0.0001), np.log(0.01)),
    'activation_hidden': 'relu',
    'activation_output': 'sigmoid'
}

In [35]:
import nnet_survival

In [36]:
def train_fine_tuning(params):
    print('Params testing: ', params)
    print('\n')

    # Prevent GPU memory allocation problems
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    K.set_session(tf.Session(config=config))

    from keras.models import load_model

    non_brca_model = load_model('keras-models/MODEL_nonBRCA_all_data_random3.h5', custom_objects={'loss': nnet_survival.surv_likelihood(n_intervals)})

    import pickle
    with open("keras-models/non_brca_random3.pkl", 'rb') as f:
        [trials_non_brca, best_non_brca] = pickle.load(f)

    if trials_non_brca.best_trial['result']['params']['dense_choice']['layers'] == 'two':
        pop_layers = 6
    elif trials_non_brca.best_trial['result']['params']['dense_choice']['layers'] == 'three':
        pop_layers = 9
        
    for i in range(pop_layers):
        non_brca_model.pop()    
    
    for layer in non_brca_model.layers:
         layer.trainable = False
            
    model = Sequential()
    
    model.add(non_brca_model)

    if params['choice']['layers']== 'two':
        # Dense layer 1 ---
        model.add(Dense(params['choice']['units_2dense1']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation_hidden']))
        model.add(Dropout(params['choice']['dropout_2dense1']))
    elif params['choice']['layers']== 'three':
        # Dense layer 1 ---
        model.add(Dense(params['choice']['units_3dense1']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation_hidden']))
        model.add(Dropout(params['choice']['dropout_3dense1']))
        # Dense layer 2 ---
        model.add(Dense(params['choice']['units_3dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation_hidden']))
        model.add(Dropout(params['choice']['dropout_3dense2']))
        
    # Output layer ---
    model.add(Dense(params['units_output'], activation=params['activation_output']))

    optimizer = Adam(lr=params['lr'])
    
    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizer)
    
    #model.summary()
    
    batch = params['batch_size']
    n_epoch = params['nb_epochs']

    hist_c_index = CIndex(X_brca_train,
                    brca_os.iloc[train_index]['OS.time'],
                    brca_os.iloc[train_index]['OS'],
                    X_brca_val,
                    brca_os.iloc[val_index]['OS.time'],
                    brca_os.iloc[val_index]['OS'],
                    filepath = 'keras-models/common_best_fine-tuning_random3_sigmoid.h5')

    callbacks = [hist_c_index]    
    
    ## Fit model using training data
    start = timer()
    model.fit(X_brca_train, 
              y_brca_train, 
              batch_size=batch, 
              epochs=n_epoch, 
              validation_data = [X_brca_val, y_brca_val],
              callbacks=callbacks,
              verbose=0)
    run_time = timer() - start
    
    predictions = model.predict_proba(X_brca_test,verbose=0)
    twoyr_surv=np.cumprod(predictions[:,0:np.nonzero(breaks>365*2)[0][0]], axis=1)[:,-1]
    score = concordance_index(brca_os.iloc[test_index]['OS.time'], twoyr_surv, brca_os.iloc[test_index]['OS'])

    # Return loss
    return {'loss': score*(-1),
            'params': params,
            'train_time': run_time,
            'status': STATUS_OK, 
            'rounds':hist_c_index.best_rounds}

In [37]:
from timeit import default_timer as timer
from multiprocessing import Pool
from hyperopt import STATUS_OK

def objective_fine_tuning(params):
    from sklearn.metrics import roc_auc_score
    
    from keras.models import Sequential
    from keras.layers import Input, Dense, Conv2D, MaxPool2D, Activation, Dropout, Flatten
    from keras import regularizers, optimizers
    from keras.layers.normalization import BatchNormalization
    from keras.callbacks import EarlyStopping

    from keras.regularizers import l1
    from keras.optimizers import Adam
       
    with Pool(1) as p:
            trial = p.apply(train_fine_tuning, args=(params, ))

    return trial

In [38]:
trials_fine_tuning = Trials()

best_fine_tuning = fmin(fn=objective_fine_tuning, space=space_fine_tuning, algo=tpe.suggest, trials=trials_fine_tuning, max_evals=100)

print (best_fine_tuning)
print (trials_fine_tuning.best_trial)

Params testing:                                      
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 16, 'choice': {'dropout_2dense1': 0.75, 'layers': 'two', 'units_2dense1': 16}, 'lr': 0.0006486092707922283, 'nb_epochs': 100, 'units_output': 39}
  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 1                             
Validation C-Index: 0.522807 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 64, 'choice': {'dropout_2dense1': 0.5, 'layers': 'two', 'units_2dense1': 2

`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 10                                                      
Validation C-Index: 0.652632 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 16, 'choice': {'dropout_3dense1': 0.5, 'dropout_3dense2': 0.75, 'layers': 'three', 'units_3dense1': 64, 'units_3dense2': 64}, 'lr': 0.0007533347115243062, 'nb_epochs': 100, 'units_output': 39}
 10%|█         | 10/100 [01:54<12:50,  8.56s/it, best loss: -0.5517241379310345]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 3                        

 19%|█▉        | 19/100 [03:37<15:32, 11.51s/it, best loss: -0.5984084880636604]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 3                                                        
Validation C-Index: 0.601170 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 32, 'choice': {'dropout_3dense1': 0.25, 'dropout_3dense2': 0.5, 'layers': 'three', 'units_3dense1': 256, 'units_3dense2': 256}, 'lr': 0.0017823792766317455, 'nb_epochs': 100, 'units_output': 39}
 20%|██        | 20/100 [03:45<14:03, 10.54s/it, best loss: -0.5984084880636604]WARNING:tensorflow:Fr

`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 6                                                        
Validation C-Index: 0.615984 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 32, 'choice': {'dropout_3dense1': 0.75, 'dropout_3dense2': 0.25, 'layers': 'three', 'units_3dense1': 32, 'units_3dense2': 32}, 'lr': 0.00114578426936509, 'nb_epochs': 100, 'units_output': 39}
 29%|██▉       | 29/100 [05:13<11:33,  9.76s/it, best loss: -0.6344827586206897]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 2                        

{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 64, 'choice': {'dropout_2dense1': 0.25, 'layers': 'two', 'units_2dense1': 64}, 'lr': 0.004390892290117097, 'nb_epochs': 100, 'units_output': 39}
 38%|███▊      | 38/100 [06:54<11:27, 11.09s/it, best loss: -0.6344827586206897]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 18                                                       
Validation C-Index: 0.621832 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 128, 'choice': {'layers': 'one'}, 'lr': 0.0004829225332177939, 'nb_epochs':

...

 57%|█████▋    | 57/100 [10:49<09:01, 12.59s/it, best loss: -0.6344827586206897]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 8                                                        
Validation C-Index: 0.524366 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 32, 'choice': {'dropout_3dense1': 0.75, 'dropout_3dense2': 0.25, 'layers': 'three', 'units_3dense1': 128, 'units_3dense2': 256}, 'lr': 0.001153153025826057, 'nb_epochs': 100, 'units_output': 39}
 58%|█████▊    | 58/100 [10:59<08:22, 11.96s/it, best loss: -0.6344827586206897]WARNING:tensorflow:Fr

`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 17                                                       
Validation C-Index: 0.608967 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 32, 'choice': {'dropout_3dense1': 0.25, 'dropout_3dense2': 0.5, 'layers': 'three', 'units_3dense1': 256, 'units_3dense2': 256}, 'lr': 0.0017185243365649087, 'nb_epochs': 100, 'units_output': 39}
 67%|██████▋   | 67/100 [12:45<06:04, 11.06s/it, best loss: -0.6344827586206897]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 10                    


Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 128, 'choice': {'dropout_3dense1': 0.25, 'dropout_3dense2': 0.5, 'layers': 'three', 'units_3dense1': 256, 'units_3dense2': 256}, 'lr': 0.0034927646627512334, 'nb_epochs': 100, 'units_output': 39}
 76%|███████▌  | 76/100 [14:37<05:06, 12.78s/it, best loss: -0.6347480106100796]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 3                                                        
Validation C-Index: 0.538791 

Params testing:                                                                 
{'activation_hidde

 85%|████████▌ | 85/100 [16:07<02:31, 10.13s/it, best loss: -0.6347480106100796]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 5                                                        
Validation C-Index: 0.524756 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 32, 'choice': {'dropout_2dense1': 0.75, 'layers': 'two', 'units_2dense1': 256}, 'lr': 0.0012273369818516172, 'nb_epochs': 100, 'units_output': 39}
 86%|████████▌ | 86/100 [16:16<02:18,  9.89s/it, best loss: -0.6347480106100796]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_

`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 3                                                        
Validation C-Index: 0.571150 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 32, 'choice': {'dropout_3dense1': 0.75, 'dropout_3dense2': 0.25, 'layers': 'three', 'units_3dense1': 16, 'units_3dense2': 64}, 'lr': 0.0031946401081116974, 'nb_epochs': 100, 'units_output': 39}
 95%|█████████▌| 95/100 [18:47<01:09, 13.85s/it, best loss: -0.6347480106100796]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Best number of rounds: 4                      

In [39]:
import pickle

with open("keras-models/common_hyper_opt_fine-tuning_random3_sigmoid.pkl", 'wb') as f:
    pickle.dump([trials_fine_tuning, best_fine_tuning], f)